# Simulate Homologous Recombination

In [2]:
from Bio import SeqIO, Align
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

## read fasta

In [3]:
def read_fasta(file_path):
    # Reads a FASTA file and returns a dictionary containing the sequence names as keys and sequences as values.
    sequences = {}
    current_sequence_name = None
    current_sequence = ''

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                # If a new sequence is encountered, store the current one (if any) and start a new sequence
                if current_sequence_name is not None:
                    sequences[current_sequence_name] = current_sequence
                current_sequence_name = line[1:]  # Remove the ">" symbol from the sequence name
                current_sequence = ''
            else:
                current_sequence += line

        # Store the last sequence after reaching the end of the file
        if current_sequence_name is not None:
            sequences[current_sequence_name] = current_sequence

    return sequences

 
# read the SD36051-001 sequence file
file_path_original = "/mnt/d/Github/XFORM_test/input/SD36051-001.fasta"
fasta_sequences_original = read_fasta(file_path_original)

# Accessing individual sequences by their names
sequence_name = "1"
sequence_original = fasta_sequences_original.get(sequence_name, None)
if sequence_original:
    print(f"Sequence name: {sequence_name}\nSequence: {sequence_original[:1000]}\n")
    print(f"Sequence length: {len(sequence_original)}\n")
else:
    print(f"Sequence '{sequence_name}' not found in the FASTA file.")   

# read the add-4353 sequence file
file_path_donor = "/mnt/d/Github/XFORM_test/input/add-4353.fasta"
fasta_sequences_donor = read_fasta(file_path_donor)

# Accessing individual sequences by their names
sequence_name2 = "ADD-4353"
sequence_donor = fasta_sequences_donor.get(sequence_name2, None)
if sequence_donor:
    print(f"Sequence name: {sequence_name2}\nSequence: {sequence_donor[2766:3266]}\n")
    print(f"Sequence length: {len(sequence_donor)}\n")
else:
    print(f"Sequence '{sequence_name2}' not found in the FASTA file.")

Sequence name: 1
Sequence: GTGTCACTTTCGCTTTGGCAGCAGTGTCTTGCCCGATTGCAGGATGAGTTACCAGCCACAGAATTCAGTATGTGGATACGCCCATTGCAGGCGGAACTGAGCGATAACACGCTGGCCCTGTACGCGCCAAACCGTTTTGTCCTCGATTGGGTACGGGACAAGTACCTTAATAATATCAATGGACTGCTAACCAGTTTCTGCGGAGCGGATGCCCCACAGCTGCGTTTTGAAGTCGGCACCAAACCGGTGACGCAAACGCCACAAGCGGCAGTGACGAGCAACGTCGCGGCCCCTGCACAGGTGGCGCAAACGCAGCCGCAACGTGCTGCGCCTTCTACGCGCTCAGGTTGGGATAACGTCCCGGCCCCGGCAGAACCGACCTATCGTTCTAACGTAAACGTCAAACACACGTTTGATAACTTCGTTGAAGGTAAATCTAACCAACTGGCGCGCGCGGCGGCTCGCCAGGTGGCGGATAACCCTGGCGGTGCCTATAACCCGTTGTTCCTTTATGGCGGCACGGGTCTGGGTAAAACTCACCTGCTGCATGCGGTGGGTAACGGCATTATGGCGCGCAAGCCGAATGCCAAAGTGGTTTATATGCACTCCGAGCGCTTTGTTCAGGACATGGTTAAAGCCCTGCAAAACAACGCGATCGAAGAGTTTAAACGCTACTACCGTTCCGTAGATGCACTGCTGATCGACGATATTCAGTTTTTTGCTAATAAAGAACGATCTCAGGAAGAGTTTTTCCACACCTTCAACGCCCTGCTGGAAGGTAATCAACAGATCATTCTCACCTCGGATCGCTATCCGAAAGAGATCAACGGCGTTGAGGATCGTTTGAAATCCCGCTTCGGTTGGGGACTGACTGTGGCGATCGAACCGCCAGAGCTGGAAACCCGTGTGGCGATCCTGATGAAAAAGGCCGACGAAAACGACATTCGTTTGCCGGGCGAAGTGGCGTTCTTTA

## Read Genebank

In [6]:
from Bio import SeqIO


def get_upstream_downstream_sequences(sequence, upstream_location, downstream_location):
    # Extracts and returns the upstream and downstream sequences from the main sequence.
   
    upstream_start = upstream_location.start
    upstream_end = upstream_location.end
    downstream_start = downstream_location.start
    downstream_end = downstream_location.end

    upstream_sequence = sequence[upstream_start:upstream_end]
    downstream_sequence = sequence[downstream_start:downstream_end]

    return upstream_sequence, downstream_sequence

def read_genbank(file_path):
    # Reads a GenBank file and returns a dictionary containing the sequence names as keys and a tuple of
    # (sequence, features, length, upstream_location, downstream_location) as values.

    sequences_with_info = {}

    with open(file_path, 'r') as file:
        for record in SeqIO.parse(file, 'genbank'):
            sequence = str(record.seq)
            features = record.features
            length = len(sequence)
            
            # Find the first and last feature locations and save them as upstream and downstream locations
            upstream_location = features[0].location
            downstream_location = features[-1].location
            
            # Save the information in the dictionary
            sequences_with_info[record.id] = (sequence, features, length, upstream_location, downstream_location)

    return sequences_with_info

# read the add-4353.gb file
file_path_genbank = "/mnt/d/Github/XFORM_test/input/add-4353.gb"
genbank_sequences = read_genbank(file_path_genbank)

# Process and print information about all sequences
for sequence_name, data in genbank_sequences.items():
    sequence, features, length, upstream_location, downstream_location = data
    print(f"Sequence name: {sequence_name}")
    print(f"Sequence length: {length} base pairs")
    print(f"Upstream location: {upstream_location}")
    print(f"Downstream location: {downstream_location}")

    # Extract and print upstream and downstream sequences
    upstream_sequence, downstream_sequence = get_upstream_downstream_sequences(sequence, upstream_location, downstream_location)
    print(f"Upstream sequence: {upstream_sequence}")
    print(f"Downstream sequence: {downstream_sequence}")

    print("Features:")
    for feature in features:
        print(f"\tFeature type: {feature.type}")
        print(f"\tLocation: {feature.location}")
        print(f"\tQualifiers: {feature.qualifiers}")


Sequence name: ADD-4353
Sequence length: 3266 base pairs
Upstream location: [0:502](+)
Downstream location: [2766:3266](+)
Upstream sequence: CCGCCACGCCGCCGCCGTTAAAGCCAACATCCATCCAGCCCACATCATCCAGCAAGAAAACAACCACGTTCGGTTTCTTACCGGTTTTTTTCTCAAGTTCTGCCAGCTTCTGCTGGGTTTCTTTGTCTTGTGCAGGATGCTGCATCACCGGCATCATGTTGTCGGCAATAGTGGTCGCCGGTTTAACCAGATACTGGTTTGGGTGATCGTATCCGGCAAAGCCTTTGCGTGCGGTGGCGGTTGACGGGGTATCTGCTGCGTTGGCCATGAGTGGAAGAGCGGCGGCGACAGCTACAACAAGAAGTTTGGGTGAAAACGAAAATTCCATGCAAAATGCTCCGGTTTCATGTCGTCAAAATGTTGACGTAATTAAGCATTGATAATTGATAATTGAGATCCCTCTCCCTGACAGGATGATTGCATAAATAATAGTGATGAAAATAAATTATTTATTTATCCAGAAAATGAATTGGAAAATCAGGAGAGCGTTTTCAATCCTACC
Downstream sequence: AGGCGGTGATGTTATATCGCGTTGATTATTGATGCTGTTTTTAGTTTTAACGGCAATTAATATATATGTTATTAATTGAATGAATTTTATTATTCATTATATATATGTGTAGAATCGTGCGCAGGAGAAATATTCACTCAGGAAGTTATTACTCAGGAAGCAAAGAGGATTACAGAATTATCTCATAACAAGTGTTAAGGGATGTTATTTCCCAGTTCTCTGTGGCATAATAAACGAGTAGATGCTCATTCTATCTCTTATGTTCGCCTTAGTGCCTCATAAACTCCGGAATGACGCAGAGCCGTTTACGGTGCTTATCGTCCACTGACAGATG

## Search the location

In [48]:

def get_complement(dna_sequence):
    # Get the complementary sequence of a DNA sequence.

    # Define the translation table to replace each nucleotide with its complement
    complement_table = str.maketrans("ATCG", "TAGC")
    
    # Use the translate method to get the complementary sequence
    complementary_sequence = dna_sequence.translate(complement_table)
    
    return complementary_sequence


def search_upstream_downstream_in_sequence(main_sequence, upstream_sequence, downstream_sequence, cutsite_sequence):
    # Searches for the location of upstream and downstream sequences in the main sequence.

    def find_sequence_in_main(sequence):
        seq_variants = [sequence, get_complement(sequence), sequence[::-1], get_complement(sequence[::-1])]
        for variant_type, seq_type in enumerate(seq_variants, start=1):
            index = main_sequence.find(seq_type)
            if index != -1:
                print(f"sequence (or its variant) found at index {index}:{index+len(sequence)}")
                return index, variant_type
        print("sequence and its variants not found")
        return -1, 0
        
    print("Upstream ", end='')
    upstream_result = find_sequence_in_main(upstream_sequence)
    print("Downstream ", end='')
    downstream_result = find_sequence_in_main(downstream_sequence)
    print("Cut site ", end='')
    find_sequence_in_main(cutsite_sequence)\
        
    if upstream_result[1] != downstream_result[1]:
        return KeyError("Upstream and downstream sequences are not the same type. Please check the sequences.")
    
    return (upstream_result[0], upstream_result[0] + len(upstream_sequence)), (downstream_result[0], downstream_result[0] + len(downstream_sequence)), upstream_result[1]


cutsite_sequence = "tctggcgcaggtgatatgta"
location_in_sequence= search_upstream_downstream_in_sequence(sequence_original.upper(), upstream_sequence.upper(), downstream_sequence.upper(), cutsite_sequence.upper())

print(location_in_sequence)

Upstream sequence (or its variant) found at index 4418691:4419193
Downstream sequence (or its variant) found at index 4418171:4418671
Cut site sequence (or its variant) found at index 4418671:4418691
((4418691, 4419193), (4418171, 4418671), 4)


## Adding the Donor DNA

In [54]:
def homologous_recombination(target_seq, insert_seq, location_in_sequence):
    # Convert sequences to Python strings
    target_seq_str = str(target_seq)
    insert_seq_str = str(insert_seq)

    # get the positions of upstream and downstream homologies in the location information handle the special case
    upstream_location_start = location_in_sequence[0][0]
    upstream_location_end = location_in_sequence[0][1]
    downstream_location_start = location_in_sequence[1][0]
    downstream_location_end = location_in_sequence[1][1]
    match_type = location_in_sequence[2]
    
    # Perform the replacement (homologous recombination)
    if match_type ==1:
        new_sequence = target_seq_str[:upstream_location_start] + insert_seq_str + target_seq_str[downstream_location_end:]
    elif match_type ==2:
        new_sequence = target_seq_str[:upstream_location_start] + get_complement(insert_seq_str) + target_seq_str[downstream_location_end:]
    elif match_type ==3:
        new_sequence = target_seq_str[:downstream_location_start] + insert_seq_str[::-1] + target_seq_str[upstream_location_end:]
    elif match_type ==4:
        new_sequence = target_seq_str[:downstream_location_start] + get_complement(insert_seq_str[::-1]) + target_seq_str[upstream_location_end:]
    else:
        return KeyError("Unknown ERROR")
    
    return new_sequence

new_sequence = homologous_recombination(sequence_original, sequence_donor, location_in_sequence)
print(len(new_sequence))

4523393
